In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
employ_file= "DP_LIVE_Employdata.csv"
GDP_file="DP_LIVE_GDPdata.csv"

In [3]:
employ_df = pd.read_csv(employ_file)
employ_df = employ_df.loc[employ_df['MEASURE'] == 'PC_WKGPOP'] #removing the measure = THND_PER; keeping only 0-100 perc. rate
employ_df.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
163,AUS,EMP,MEN,PC_WKGPOP,A,1979,82.10677,NaN
164,AUS,EMP,MEN,PC_WKGPOP,A,1980,82.25813,NaN
165,AUS,EMP,MEN,PC_WKGPOP,A,1981,82.32400,NaN
166,AUS,EMP,MEN,PC_WKGPOP,A,1982,80.50681,NaN
167,AUS,EMP,MEN,PC_WKGPOP,A,1983,76.92610,NaN


In [4]:
GDP_df=pd.read_csv(GDP_file)
GDP_df.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,GDP,TOT,MLN_USD,A,1960,25036.1668,NaN
1,AUS,GDP,TOT,MLN_USD,A,1961,25326.3822,NaN
2,AUS,GDP,TOT,MLN_USD,A,1962,27913.2057,NaN
3,AUS,GDP,TOT,MLN_USD,A,1963,30385.7232,NaN
4,AUS,GDP,TOT,MLN_USD,A,1964,32695.8783,NaN


In [5]:
GDP_new_df=GDP_df[["LOCATION","INDICATOR","SUBJECT","TIME","Value"]]
GDP_new_df.head()

,LOCATION,INDICATOR,SUBJECT,TIME,Value
0,AUS,GDP,TOT,1960,25036.1668
1,AUS,GDP,TOT,1961,25326.3822
2,AUS,GDP,TOT,1962,27913.2057
3,AUS,GDP,TOT,1963,30385.7232
4,AUS,GDP,TOT,1964,32695.8783


In [6]:
GDP_new_df = GDP_new_df.rename(columns={"Value":"GDP Value"})
GDP_new_df = GDP_new_df[['LOCATION', 'TIME', 'GDP Value']]
GDP_new_df = GDP_new_df.rename(columns={"LOCATION":"Country","TIME":"Year"})

GDP_new_df.head()

,Country,Year,GDP Value
0,AUS,1960,25036.1668
1,AUS,1961,25326.3822
2,AUS,1962,27913.2057
3,AUS,1963,30385.7232
4,AUS,1964,32695.8783


In [7]:
employ_new_df=employ_df[["LOCATION","INDICATOR","SUBJECT","TIME","Value"]]
employ_new_df=employ_new_df.rename(columns={"LOCATION":"Country","TIME":"Year","Value":"Emp Value"})
employ_new_df.head()

,Country,INDICATOR,SUBJECT,Year,Emp Value
163,AUS,EMP,MEN,1979,82.10677
164,AUS,EMP,MEN,1980,82.25813
165,AUS,EMP,MEN,1981,82.32400
166,AUS,EMP,MEN,1982,80.50681
167,AUS,EMP,MEN,1983,76.92610


#### Cleaning

In [8]:
# Dropping the values where subject != Total (Men; Women)
# GDP_new_df = GDP_new_df.loc[GDP_new_df['SUBJECT'] == "TOT"]

employ_new_df = employ_new_df.loc[employ_new_df['SUBJECT'] == "TOT"]
employ_new_df.head()

,Country,INDICATOR,SUBJECT,Year,Emp Value
203,AUS,EMP,TOT,1979,64.38335
204,AUS,EMP,TOT,1980,65.17445
205,AUS,EMP,TOT,1981,65.36742
206,AUS,EMP,TOT,1982,64.16441
207,AUS,EMP,TOT,1983,61.94022


In [9]:
employ_new_df = employ_new_df[['Country', 'Year', 'Emp Value']]


In [10]:
#Employement has 2016-Q2 & 2016-12 data - need to get grouped, averaged out and replaced on yearly basis

employ_new_df['Year'] = pd.to_datetime(employ_new_df['Year']).apply(lambda x : pd.Period(x, freq='M').year)

#employ_new_df.groupby(['LOCATION'], ['TIME']).mean()
#employ_new_df.groupby(['LOCATION','TIME']).reset_index()
#employ_new_df = employ_new_df.groupby(('LOCATION', 'TIME'], as_index=False).agg({"Value": "sum"})

# employ_new_df= employ_new_df.groupby(['LOCATION','TIME'], as_index=False).agg({'Value':'mean'})
employ_new_df.head()


,Country,Year,Emp Value
203,AUS,1979,64.38335
204,AUS,1980,65.17445
205,AUS,1981,65.36742
206,AUS,1982,64.16441
207,AUS,1983,61.94022


In [11]:
employ_new_df = employ_new_df.loc[employ_new_df['Year'] >= 2000]
GDP_new_df = GDP_new_df.loc[GDP_new_df['Year'] >= 2000]



In [12]:
GDP_new_df = GDP_new_df.groupby(['Country', 'Year'], as_index=False).agg({"GDP Value": "mean"})
GDP_new_df.head()

,Country,Year,GDP Value
0,ARG,2004,247418.443175
1,ARG,2005,277710.485069
2,ARG,2006,308920.358148
3,ARG,2007,345624.182414
4,ARG,2008,366612.679462


In [13]:
employ_new_df = employ_new_df.groupby(['Country', 'Year'], as_index=False).agg({"Emp Value": "mean"})
#employ_new_df

In [14]:
#combined = pd.merge(GDP_new_df, employ_new_df, 
#                    how = 'inner', left_on= ['Year', 'Country'], right_on = ['Year', 'Country'])

combined_df= pd.merge(GDP_new_df, employ_new_df, on = ['Year', 'Country'], how='inner')
combined_df.drop_duplicates(keep=False)
combined_df

,Country,Year,GDP Value,Emp Value
0,AUS,2000,2.830109e+05,69.085338
1,AUS,2001,2.985813e+05,69.009075
2,AUS,2002,3.150211e+05,69.332712
3,AUS,2003,3.336445e+05,69.985809
4,AUS,2004,3.546693e+05,70.286239
5,AUS,2005,3.766379e+05,71.520062
6,AUS,2006,4.060950e+05,72.112506
7,AUS,2007,4.323268e+05,72.796190
8,AUS,2008,4.460390e+05,73.184184
9,AUS,2009,4.721421e+05,72.041994
